<a href="https://colab.research.google.com/github/AleemKhan12/AleemKhan12/blob/main/Attendance_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
#Revision notes V0.5: Mentee counts no longer include program directors


# CONNECTING TO GOOGLE DRIVE

In [2]:
#connect colab to drive

from google.colab import drive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import gspread
drive.mount('/content/gdrive')
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())


Mounted at /content/gdrive


In [3]:
#Imports
import pandas as pd
from datetime import date, timedelta
import pytz
from numpy.core.numeric import NaN
import datetime
import math


#IMPORTANT GLOBAL VARIABLES

In [4]:
mentor_email = 'mentor@villagebookbuilders.org' #Mentor program email address
oracle_data       = 0                           #Data read from Oracle Database SQL Query
google_data       = 0                           #Data read from Google Audit Log
active_sessions   = 0                           #Active sessions recorded in Oracle db based on end date
total_mentors     = 0                           #List of all mentors in oracle db
active_mentors    = 0                           #Mentors that joined atleast one session 
mentor_programs   = []                          #Names of all mentor programs stored in oracle database
mentors_not_registered = []                     #Names of all active mentors not logged in oracle database
time_zone = 'America/Port_of_Spain'             #google_data['Date'][1][len(google_data['Date'][1])-3:] #TODO: change this hard coded time zone
program_director_name = ['adeiso ghana', 'ahero kenya', 'biwi, malawi vbb mentor program', 'charity centre, zambia vbb mentor program', 'chebosi kenya', 'kadzakalowa malawi', 'maragoli kenya', 'mukono uganda', 'vima ghana'] #Actor names of program director accounts

# READING DATA SOURCES

In [5]:
#Read Data Sources as pandas data frames

#Oracle Data
oracle_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/export (1).csv' 
oracle_data = pd.read_csv(oracle_path)

#Google Data
#google_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Google Audit Log/Jan 2022/Copy of meet_logs_1643126905783 - meet_logs_1643126905783.csv'
google_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Google Audit Log/Feb 2022/2 22 2022.csv'
google_data = pd.read_csv(google_path)


#Mentee Computer Emails
mentee_email_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/mentee email/Copy of Village Library Information - computer email.csv'
mentee_computer_emails = pd.read_csv(mentee_email_path)





# CLEANING DATA

In [6]:

#Oracle Data
oracle_data = oracle_data.dropna(subset=['HANGOUTS_LINK', 'FIRST_NAME']) #remove all empty sessions

#Google Data
google_data = google_data.dropna(subset=['Date', 'Meeting code', 'Participant Identifier', 'Organizer email', 'Duration', 'Actor name']) #clean google data
google_data = google_data[google_data['Duration'] != 0] #drop all calls with 0 duration

#Mentee Email data
mentee_computer_emails = mentee_computer_emails.dropna()

In [7]:
google_data.shape

(13170, 64)

In [8]:
#Keep all google data whose organizor email is mentor@villagebookbuilders.org
# mentor_google_data = google_data.loc[google_data['Organizer email'] == mentor_email]
# mentor_google_data.shape
google_data = google_data.loc[google_data['Organizer email'] == mentor_email]
google_data.shape

(9727, 64)

In [9]:
#DATE CONVERSIONS
#Only run this once. Running more times will cause an error. To fix re-read data sources
def conv_to_datetime(a):
  if isinstance(a, datetime.date):
    return a
  try:
    date = datetime.datetime.strptime(a, '%m/%d/%Y %H:%M')
    date = date.replace(tzinfo = pytz.timezone('utc'))
    return date
  except:
    try:
      utc_timezone = pytz.timezone("utc")
      date = datetime.datetime.strptime(a, '%b %d, %Y, %I:%M:%S %p')
      date = date.replace(tzinfo = pytz.timezone(time_zone)) 
      date = date.astimezone(utc_timezone) #convert all times to UTC
      return  date
    except:
      date = datetime.datetime.strptime('', '')
      date = date.replace(tzinfo = pytz.timezone('utc'))
      return date

#converting oracle dates to datetime objs
oracle_data['END_DATE'] = oracle_data['END_DATE'].apply(conv_to_datetime)
oracle_data['START_DATE'] = oracle_data['START_DATE'].apply(conv_to_datetime)
oracle_data

google_data['Date'] = google_data['Date'].apply(lambda x: x[:len(x)-4]) #remove time zone
google_data['Date'] = google_data['Date'].apply(conv_to_datetime)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
#Total active sessions
# today_date = datetime.datetime.today()
# today_date = today_date.replace(tzinfo = pytz.timezone(time_zone))
# active_sessions = oracle_data.loc[(oracle_data['HANGOUTS_LINK'] == oracle_data['HANGOUTS_LINK']) & (oracle_data['END_DATE'] >= today_date)] #could use dropna() function to get rid of NaN's
# active_sessions.shape

#oracle_data['HANGOUTS_LINK'][1]


In [11]:
#Get Mentor Full Name

#active_sessions['Actor name'] = active_sessions['FIRST_NAME'] +' '+ active_sessions['LAST_NAME']
oracle_data['Actor name'] = oracle_data['FIRST_NAME'] +' '+ oracle_data['LAST_NAME']
total_mentors = sorted(oracle_data['Actor name'].unique().astype(str))
total_mentors = [x.lower() for x in total_mentors]


In [12]:
#get all mentors that joined atleast one meet
active_mentors = sorted(google_data['Actor name'].unique())
active_mentors = [participant.lower() for participant in active_mentors]
#active_mentors #ordered LIST of mentor names

In [13]:
#Convering MSM to Weekday and time  UTC

def conv_to_weekday(a):
  weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
  try:
    a = int(a)
    day_of_week = weekdays[int(a/(60*24))] + ' ' + str(int((a%(60*24))/60)) + ':' + str((a%(60*24))%60) + ' UTC'
    day_of_week = int(a/(60*24))
    return day_of_week
  except: 
    pass

#MSM = active_sessions['MSM']
#active_sessions['SESSION_DATE'] = active_sessions['MSM'].apply(conv_to_weekday)
oracle_data['SESSION_DATE'] = oracle_data['MSM'].apply(conv_to_weekday)

In [14]:
#lower Actor name in Google Audit Log Data for accurate comparisons
google_data['Actor name'] = google_data['Actor name'].apply(lambda x: x.lower())
oracle_data['Actor name'] = oracle_data['Actor name'].astype(str) #convert NaN's to string
oracle_data['NAME'] = oracle_data['NAME'].astype(str) #convert NaN's to string
oracle_data['Actor name'] = oracle_data['Actor name'].apply(lambda x: x.lower())
oracle_data['NAME'] = oracle_data['NAME'].apply(lambda x: x.lower())

#STAT TRACKING (MENTORS IN DATABASE)

In [46]:
#Get start, end dates and today's date
dates_list = sorted(google_data['Date'].tolist())
#start, end = dates_list[0], dates_list[-1]
end, start = datetime.datetime(2022, 2, 23).astimezone(pytz.timezone("utc")), datetime.datetime(2022, 1, 1).astimezone(pytz.timezone("utc"))
today_date = datetime.datetime.today().replace(tzinfo = pytz.timezone(time_zone))


In [47]:
count = 0 #how many mentor to print. Mainly for testing purposes
#start, end = datetime.datetime(2021, 7, 29, tzinfo = pytz.timezone(time_zone)), datetime.datetime(2022, 1, 25, tzinfo = pytz.timezone(time_zone))

#get number of weeks in start and end dates
max_size = 0
for i in range(0, 7):
  days = (start + timedelta(days=i) for i in range((end - start).days + 1))
  l = [d.strftime("%Y-%m-%d") for d in days if d.weekday() in [i]]
  if len(l)>max_size:
    max_size = len(l)


In [ ]:
#Get attendance of each mentor in oracle database

attendance_data = []
duration_data = []

for mentor in total_mentors:
  # if count == 1000:
  #   break
  # count += 1

  #Get participant data 
  participant_oracle_data = oracle_data.loc[oracle_data['Actor name']  == mentor] #read oracle data for participant
  participant_google_data = google_data.loc[google_data['Actor name'] == mentor]  #read google data for participant

  #Get list of dates that they joined sessions
  participant_google_data['String_Date'] = participant_google_data['Date'].apply(lambda x: str(x)[0:10])
  google_dates = participant_google_data.drop_duplicates(subset=['String_Date'])['String_Date'].tolist()#participant_google_data


  #Check attendance and duration for each session in oracle database
  for num, row in participant_oracle_data.iterrows():
    days = (start + timedelta(days=i) for i in range((end - start).days + 1))
    l = [d.strftime("%Y-%m-%d") for d in days if d.weekday() in [int(row['SESSION_DATE'])]]
    attendance_list = [0] * max_size #every array will be the same size regarless of the day of week that sessions are held
    duration_list = [0] * max_size
    index = 0;
    for date in l:
      if(date in google_dates):
        #Record attendance
        attendance_list[index] = 1

        #Record total time spent in meeting
        duration = 0
        duration = participant_google_data.loc[participant_google_data['String_Date'] == date]['Duration'].sum() #this will sum total duration on given day regardless of meeting code
        duration_list[index] = duration

      #increment index
      index = index+1

    #Calculating attendance and duration Stats
    attendance_stats = (sum(attendance_list)/len(attendance_list))*100
    duration_stats = sum(duration_list)
    attendance_list.append(attendance_stats)
    duration_list.append(duration_stats)

    #Insert meta data into lists
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    #attendance list
    attendance_list.insert(0, row['Actor name'])
    attendance_list.insert(1, row['NAME'].lower())
    attendance_list.insert(2, ('Y' if row['END_DATE'] < today_date else 'N'))
    attendance_list.insert(3, weekdays[row['SESSION_DATE']])
    attendance_list.insert(4, row['HANGOUTS_LINK'][-12:].replace('-','').upper())
    attendance_list.insert(5, row['VBB_EMAIL'])
    attendance_list.insert(6, 'Mentor')

    #duration list
    duration_list.insert(0, row['Actor name'])
    duration_list.insert(1, row['NAME'].lower())
    duration_list.insert(2, ('Y' if row['END_DATE'] < today_date else 'N'))
    duration_list.insert(3, weekdays[row['SESSION_DATE']])
    duration_list.insert(4, row['HANGOUTS_LINK'][-12:].replace('-','').upper())
    duration_list.insert(5, row['VBB_EMAIL'])
    duration_list.insert(6, 'Mentor')

    #append mentor attendance data to general list for dataframe generation
    attendance_data.append(attendance_list)
    duration_data.append(duration_list)
  

In [49]:
print(len(attendance_data), len(duration_data))

349 349


In [50]:
days = (start + timedelta(days = i*7) for i in range((end - start).days + 1))
l = [d.strftime("%V") for d in days]
unique_list = []
  # traverse for all elements
for x in l:
    # check if exists in unique_list or not
    if x not in unique_list:
        unique_list.append(x)

l = unique_list[:max_size]
print(l)

['52', '01', '02', '03', '04', '05', '06', '07']


#STAT TRACKING (MENTORS NOT IN DATABASE)

In [52]:
#Get list of active mentors not in database
s1 = oracle_data['Actor name']
s2 = google_data['Actor name']
mentors_not_registered = sorted(s2[~s2.isin(s1)].unique())
mentors_not_registered = [x for x in mentors_not_registered if x not in program_director_name]


In [53]:
len(mentors_not_registered)

177

In [ ]:
#Get list of all mentor programs
mentor_programs = sorted(oracle_data['NAME'].unique())
mentor_programs = [x.lower() for x in mentor_programs]
mentor_programs

In [22]:
#clear lists for testing
# attendance_data = []
# duration_data = []

In [55]:
#getting attendance data for mentors not in orable database
#attendance_data = []

#Get participant name
for participant_name in mentors_not_registered:
  #participant_name = mentors_not_registered[17]
  participant_google_data = google_data.loc[google_data['Actor name'] == participant_name]
  #print(participant_name)

  #Get session info
  participant_google_data['String_Date'] = participant_google_data['Date'].apply(lambda x: str(x)[0:10])
  #participant_google_data = participant_google_data.drop_duplicates(subset=['String_Date', 'Meeting code'])

  #Get mentor session info
  session_info = participant_google_data[['Meeting code', 'Date']].drop_duplicates(subset=['Meeting code'])
  session_info['Date'] = session_info['Date'].apply(lambda x: x.weekday())
  #print(session_info)

  #Get dates
  google_dates = participant_google_data['String_Date'].tolist()

  #mentee emails 
  mentee_emails = mentee_computer_emails['COMPUTER_EMAIL'].tolist()

  #Find stats for each session
  for num, session in session_info.iterrows():

    #print(day)
    days = (start + timedelta(days=i) for i in range((end - start).days + 1))
    l = [d.strftime("%Y-%m-%d") for d in days if d.weekday() in [session['Date']]]
    attendance_list = [0] * max_size #every array will be the same size regarless of the day of week that sessions are held
    duration_list = [0] * max_size
    index = 0;
    for date in l:
      if(date in google_dates):
        #track attendance
        attendance_list[index] = 1

        #Track duration
        duration = 0
        duration = participant_google_data.loc[(participant_google_data['String_Date'] == date) & (participant_google_data['Meeting code'] == session['Meeting code'])]['Duration'].sum() #this will sum total duration on given day regardless of meeting code
        duration_list[index] = duration

      index = index+1
    
    #Calculating attendance Stats
    attendance_stats = (sum(attendance_list)/len(attendance_list))*100
    duration_stats = sum(duration_list)
    attendance_list.append(attendance_stats)
    duration_list.append(duration_stats)

    
    
    #get meeting code
    meeting_code = session['Meeting code']
    #print(meeting_code)

    #get participants that have this meeting code
    mentee_info = sorted(google_data.loc[google_data['Meeting code'] == meeting_code]['Participant Identifier'].unique()) #will always return an array of at least 1
    #print(mentee_info)

    #Get program info from participant identifiers
    mentor_program = NaN
    for i in range(0, len(mentee_info)):
      if ((mentee_info) and (mentee_info[i] in mentee_emails)):
        mentor_program = mentee_computer_emails.loc[mentee_computer_emails['COMPUTER_EMAIL'] == mentee_info[i]]['PROGRAM_NAME'].tolist()[0]
        break
    #print(mentor_program)

    #Add meta data to data frame
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    #Attendance list
    attendance_list.insert(0, participant_name)
    attendance_list.insert(1, mentor_program)
    attendance_list.insert(2, 'N')
    attendance_list.insert(3, weekdays[session['Date']])
    attendance_list.insert(4, session['Meeting code']) #add column for meeting code
    
    #Get participant email
    participant_email = participant_google_data['Participant Identifier'].tolist()[0]
    attendance_list.insert(5, participant_email)
    attendance_list.insert(6, ('Mentee' if participant_email in mentee_computer_emails['COMPUTER_EMAIL'].tolist() else 'Mentor'))

    #Duration list
    duration_list.insert(0, participant_name)
    duration_list.insert(1, mentor_program)
    duration_list.insert(2, 'N')
    duration_list.insert(3, weekdays[session['Date']])
    duration_list.insert(4, session['Meeting code']) #add column for meeting code
    duration_list.insert(5, participant_google_data['Participant Identifier'].tolist()[0])
    duration_list.insert(6, ('Mentee' if participant_google_data['Participant Identifier'].tolist()[0] in mentee_computer_emails['COMPUTER_EMAIL'].tolist() else 'Mentor'))
    
    
    attendance_data.append(attendance_list)
    duration_data.append(duration_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [56]:
print(len(attendance_data), len(duration_data))

924 924


#CREATING DATAFRAME AND STORING ATTENDANCE DATA

In [57]:
#Generate dataframe column names
days = (start + timedelta(days=i) for i in range((end - start).days + 1))
column_names = ['WK'+str(n) for n in range(0, max_size)]
column_names.insert(0, 'Mentor Name')
column_names.insert(1, 'Program Name')
column_names.insert(2, 'Is Expired')
column_names.insert(3, 'Session Day')
column_names.insert(4, 'Meet Code')
column_names.insert(5, 'Participant Email')
column_names.insert(6, 'Participant Type')
column_names.append('Attendance Stats')

#create dataframes

attendance_df = pd.DataFrame(attendance_data, columns=column_names)

#change column name
column_names[-1] = 'Duration Stats'
duration_df = pd.DataFrame(duration_data, columns=column_names)

In [58]:
print(attendance_df.shape, duration_df.shape)

(924, 16) (924, 16)


In [59]:
#File names
attendance_file_name = 'all_mentor_attendance_data.csv'
duration_file_name = 'all_mentor_duration_data.csv'

#writing dataframes to csv
attendance_df.to_csv(attendance_file_name)
duration_df.to_csv(duration_file_name)

#MENTORS THAT NEED TO EXTEND THEIR END DATE

In [28]:
#get list of people that need to extend their end date
expired_participants = oracle_data.loc[(oracle_data['END_DATE'] < today_date)]
s1 = expired_participants['Actor name']
#print(oracle_data['Actor name'] in active_mentors)

In [29]:
s2 = google_data['Actor name']
# print(google_data['Actor name'])
# print(expired_participants['Actor name'])
mentors_to_renew = s1[s1.isin(s2)]
mentors_to_renew


7              aiman hanif
24       edzordzinam fugar
30               tina chow
31           nethmi perera
34      arshia chakrabarti
               ...        
1023         rachel mathai
1024         sandy granger
1045         shreyans shah
1049         ashrit ramesh
1075         sumaiya hanif
Name: Actor name, Length: 177, dtype: object

# AVAILABLE PROGRAM SLOTS 

In [30]:
#get total sessions by program
total_sessions = oracle_data['NAME'].value_counts()
total_sessions

ahero, kenya                                                                                            100
kadzakalowa, malawi                                                                                      64
maragoli, kenya                                                                                          37
charity centre, zambia                                                                                   36
mukono, uganda                                                                                           32
vima, ghana                                                                                              23
adeiso, ghana                                                                                            21
biwi, malawi                                                                                             13
jolly land, uganda                                                                                       13
high tech, uganda           

In [31]:
#get count of active sessions
#taken_sessions = active_sessions['NAME'].value_counts()
#taken_sessions

In [32]:
available_sessions = total_sessions - taken_sessions
available_sessions

NameError: ignored